# Домашнее задание № 1

In [1]:
%%capture
!pip install pymorphy3 pymystem3 razdel spacy
!python -m spacy download ru_core_news_sm

In [2]:
import hashlib
import pandas as pd
import razdel
import spacy

from collections import defaultdict
from pymorphy3 import MorphAnalyzer
from pymystem3 import Mystem
from tqdm.auto import tqdm

## Задание 1 (2 балла)

Найдите 1 любой способ сломать токенизацию на предложения функцией sentenize из библиотеки razdel. Придумайте (или найдите на каком-то корпусе) такое предложение (или несколько предложений), которое будет некорректно разобрано sentenize, но при этом будет грамматически корректным.

In [3]:
# пат.пов. = патентный поверенный
text = "Контактное лицо: пат.пов. Г.Б. Егорова."
sents = list(razdel.sentenize(text))
for sent in sents:
    print(sent.text)

Контактное лицо: пат.пов.
Г.Б. Егорова.


## 2. Токенизация Mystem vs razdel.tokenize (2 балла)


Токенизируйте текст с помощью razdel и с помощью Mystem. Найдите различия в токенизациях. Что по вашему работает лучше на приведенном тексте?

In [4]:
text = """
Вторым и третьим открытыми белыми карликами стали Сириус B и Процион B. В 1844 году директор Кёнигсбергской обсерватории Фридрих Бессель, анализируя данные наблюдений, которые велись с 1755 года, обнаружил, что Сириус, ярчайшая звезда земного неба, и Процион периодически, хотя и весьма слабо, отклоняются от прямолинейной траектории движения по небесной сфере[5]. Бессель пришёл к выводу, что у каждой из них должен быть близкий спутник. Сообщение было встречено скептически, поскольку слабый спутник оставался ненаблюдаемым, а его масса должна была быть достаточно велика — сравнимой с массой Сириуса и Проциона, соответственно.

В январе 1862 года Элвин Грэхэм Кларк, юстируя 18-дюймовый рефрактор, самый большой на то время телескоп в мире (Dearborn Telescope), впоследствии поставленный семейной фирмой Кларков в обсерваторию Чикагского университета, обнаружил в непосредственной близости от Сириуса тусклую звёздочку. Это был спутник Сириуса, Сириус B, предсказанный Бесселем[6]. А в 1896 году американский астроном Д. М. Шеберле открыл Процион B, подтвердив тем самым и второе предсказание Бесселя.

В 1915 году американский астроном Уолтер Сидней Адамс измерил спектр Сириуса B. Из измерений следовало, что его температура не ниже, чем у Сириуса A (по современным данным, температура поверхности Сириуса B составляет 25 000 K, а Сириуса A — 10 000 К), что, с учётом его в 10 000 раз меньшей, чем у Сириуса A, светимости указывает на очень малый радиус и, соответственно, высокую плотность — 106 г/см3 (плотность Сириуса ~0,25 г/см3, плотность Солнца ~1,4 г/см3).
"""

In [5]:
razdel_tokens = []
for token in razdel.tokenize(text):
    razdel_tokens.append(token.text)

Из токенизации Mystem выбросим пробелы и символы переноса строки для сравнимости

In [6]:
mystem = Mystem()
mystem_tokens = []
for token in mystem.analyze(text):
    if token["text"] not in ["\n", " "]:
        mystem_tokens.append(token["text"])

In [7]:
df = pd.DataFrame(
    {
        "razdel": pd.Series(razdel_tokens),
        "mystem": pd.Series(mystem_tokens),
    }
)
df

,razdel,mystem
0,Вторым,Вторым
1,и,и
2,третьим,третьим
3,открытыми,открытыми
4,белыми,белыми
...,...,...
288,см,г
289,3,/
290,),см3
291,.,)


Различия:

|кол-во вхождений| razdel            | mystem                            |
|--|-------------------|-----------------------------------|
|1| ✅ "18-дюймовый"  | ❌ "18", "-", "дюймовый"          |
|2| ✅ ")", ","       | ❌ "),"                           |
|3| ❌ "см", "3"      | ✅ "см3"                          |
|2| ✅ "0,25" / "1,4" | ❌ "0", ",", "25" / "1", ",", "4" |

У razdel на этом тексте токенизация лучше. Меньше как и количество вхождений плохих токенизаций (razdel 3 vs. mystem 5), так и количество типов плохих токенизаций (razdel 1 vs. mystem 3).

## 3. Лемматизация Mystem vs Pymorphy (2 балла)

Лемматизируйте текст с помощью mystem и pymorphy. Найдите различия в лемматизации. Что по вашему работает лучше на приведенном тексте?

Важно: для пайморфи используйте токенизацию из mystem, чтобы исключить влияние токенизации на результат. Анализируйте только значимые различия, а не технические особенности (не сравнивайте скорость работы и удобность интерфейса).

В майстеме убедитесь, что используется дизамбигуация.

In [8]:
# у mystem дизамбигуация используется by default
mystem_tokens = []
mystem_lemmata = []
for token in mystem.analyze(text):
    # выкидываем не только пробелы и \n, но и всю пунктуацию
    if "analysis" in token.keys():
        mystem_tokens.append(token["text"])
        if len(token["analysis"]) > 0:
            mystem_lemmata.append(token["analysis"][0]["lex"])
        else:
            # берем текст as is, если он не на русском ("B" в "Сириус B")
            mystem_lemmata.append(token["text"].lower())

In [9]:
morph = MorphAnalyzer()
pymorphy_lemmata = [
    morph.parse(token)[0].normal_form for token in mystem_tokens
]

In [10]:
df = pd.DataFrame(
    {
        "token": mystem_tokens,
        "mystem": mystem_lemmata,
        "pymorphy": pymorphy_lemmata,
    }
)

In [11]:
df[df["mystem"] != df["pymorphy"]]

,token,mystem,pymorphy
6,стали,становиться,стать
15,Кёнигсбергской,кенигсбергский,кёнигсбергский
20,данные,данные,дать
26,обнаружил,обнаруживать,обнаружить
49,пришёл,приходить,прийти
63,встречено,встречать,встретить
71,его,его,он
89,Грэхэм,грэхэм,грэхэма
105,поставленный,поставлять,поставить
113,обнаружил,обнаруживать,обнаружить


|кол-во вхождений| token                                   | mystem                                | pymorphy                            |
|--|-----------------------------------------|---------------------------------------|-------------------------------------|
|9| глаголы совершенного вида | ❌ инфинитив глагола несовершенного вида | ✅ инфинитив глагола совершенного вида |
|3| слово с буквой "ё" | ❌ лемма без "ё"                         | ✅ лемма с "ё"                         |
|1| данные*                                  | ✅ данные                                | ❌ дать                                |
|2| его*                                     | ✅ его                                   | ❌ он                                  |
|1| Грэхэм                                  | ✅ грэхэм                                | ❌ грэхэма                             |
|1| Д                                       | ✅ д                                     | ❌ далее                               |
|1| Шеберле                                 | ❌ шеберль                               | ❌ шеберл                              |
|1| тем                                     | ❌ то                                    | ❌ тем                                 |
|1| второе                                  | ✅ второй                                | ❌ второе                              |
|1| ниже                                    | ✅ низкий                                | ❌ ниже                                |
|1| меньшей                                 | ❌ меньший                               | ✅ малый                               |

*"данные" в этом контексте - существительное без формы единственного числа.

*"его" - в обоих контекстах притяжательное, а не личное местоимение.

Mystem 9 раз допустил ошибку с видом глагола. Даже если это feature, а не баг Mystem, определять леммой глагола совершенного вида глагол совершенного вида неверно, т. к. вид в русском языке ближе к словообразовательной, а не словоизменительной категории.

Победителем на этом тексте тем не менее можно считать Mystem: глаголы совершенного вида и слова с "ё" можно отфильтровать и прогнать через другой лемматизатор, тогда как среди плохо предсказуемых ошибок (не считая те, где оба лемматизатора определили лемму неверно) 6 принадлежат Pymorphy и только одна - Mystem.

## 4. Лемматизация в SpaCy (2 балла)

С помощью Spacy (модель для русского языка) лемматизируйте тот же текст. Проверьте есть ли различия с Mystem и Pymoprhy.

In [12]:
nlp = spacy.load("ru_core_news_sm")

In [13]:
# встроим в spacy токенизатор от mystem,
# чтобы исключить влияние токенизации на результат
def mystem_tokenizer(text):
    tokens = []
    for token in mystem.analyze(text):
        if "analysis" in token.keys():
            tokens.append(token["text"])
    return spacy.tokens.Doc(nlp.vocab, tokens)

nlp.tokenizer = mystem_tokenizer

In [14]:
doc = nlp(text)

spacy_lemmata = []
for sent in doc.sents:
    for token in sent:
        # приведем к нижнему регистру
        spacy_lemmata.append(token.lemma_.lower())
df["spacy"] = spacy_lemmata

Различия с Pymorphy:

In [15]:
df[df["spacy"] != df["pymorphy"]][["token", "pymorphy", "spacy"]]

,token,pymorphy,spacy
20,данные,дать,данные
22,которые,который,которые
56,них,они,них
62,было,быть,было
83,Проциона,процион,проциона
89,Грэхэм,грэхэма,грэхэм
97,то,то,тот
108,Кларков,кларк,кларков
111,Чикагского,чикагский,чикагского
128,Бесселем,бессель,бесселем


Различия с Mystem:

In [16]:
df[df["spacy"] != df["mystem"]][["token", "mystem", "spacy"]]

,token,mystem,spacy
6,стали,становиться,стать
15,Кёнигсбергской,кенигсбергский,кёнигсбергский
22,которые,который,которые
26,обнаружил,обнаруживать,обнаружить
49,пришёл,приходить,прийти
56,них,они,них
62,было,быть,было
63,встречено,встречать,встретить
71,его,его,он
83,Проциона,процион,проциона


## 5*. LSH (2 балла)

*необязательное задание чтобы получить 10 баллов

Попробуйте искать дубликаты в настоящих текстах. Например, можете взять https://github.com/mannefedov/compling_nlp_hse_course/blob/master/data/anna_karenina.txt или https://github.com/mannefedov/compling_nlp_hse_course/blob/master/data/besy_dostoevsky.txt (или любой другой корпус)

Используйте код из семинара для нахождения кандидатов в дубликаты (шинглы -> минхэш - lsh) и рассчитайте реальную меру Жаккара между полученными кандидатами. Настройте параметры k, num_hash_functions, bands так чтобы результаты получались адекватные (мера Жаккара хотя бы больше нуля).

(Можете взять 500-1000 текстов если весь корпус обрабатывается слишком долго)

Разделим корпус на предложения с помощью spacy

In [17]:
corpus = open("anna_karenina.txt").read()

In [18]:
nlp = spacy.load("ru_core_news_sm", disable=["tagger", "ner"])
doc = nlp(corpus[:1000000])
sents = [str(sent) for sent in doc.sents]
# берем только предложения длиннее 20 символов
sents = [sent for sent in sents if len(sent) > 20]
print(f"Всего {len(sents)} предложений")

Всего 9340 предложений


Код для нахождения кандидатов в дубликаты

In [19]:
def hash_string(s):
    """хеширует строку и возвращает число"""
    return int(hashlib.md5(s.encode("utf8")).hexdigest(), 16)

In [20]:
def generate_hash_functions(k):
    """генерирует k хеш-функций добавляя индекс к строке"""
    functions = []
    for i in range(k):
        functions.append(lambda x, i=i: hash_string(x + str(i)))
    return functions

In [21]:
def get_shingles(text, k=5):
    """генерирует список шинглов из строки"""
    shingles = set()
    for i in range(len(text) - k + 1):
        shingle = text[i:i + k]
        shingles.add(shingle)
    return shingles

In [22]:
def compute_minhash_signature(shingles, hash_funcs):
    """вычисляет minhash-сигнатуру для списка шинглов"""
    signature = []
    for hash_func in hash_funcs:
        min_hash = min(hash_func(shingle) for shingle in shingles)
        signature.append(min_hash)
    return signature

In [23]:
def lsh(signatures, bands):
    """Разрезает сигнатуры на куски (bands), и группирует индексы сигнатур по совпадению кусков"""
    buckets = defaultdict(list)
    band_length = len(signatures[0]) // bands

    for idx, sig in tqdm(enumerate(signatures)):
        for b in range(0, bands, band_length):
            start = b
            end = start + band_length
            band = tuple(sig[start:end])
            buckets[band].append(idx)
    return buckets

In [24]:
def find_similar_strings(strings_list, k=5, num_hashes=100, bands=20):
    """Finds similar strings using MinHash and LSH."""
    hash_funcs = generate_hash_functions(num_hashes)
    signatures = []
    shingles_list = []

    # каждый текст обрабатывается отдельно
    # находятся шинглы и рассчитываются сигнатуры
    for string in tqdm(strings_list):
        shingles = get_shingles(string, k)
        shingles_list.append(shingles)
        signature = compute_minhash_signature(shingles, hash_funcs)
        signatures.append(signature)

    # вычисляются кандидаты по кускам сигнатур
    buckets = lsh(signatures, bands)
    candidates = set()
    for bucket in buckets.values():
        if len(bucket) > 1:
            for i in bucket:
                for j in bucket:
                    if i < j:
                        candidates.add((i, j))


    return candidates

In [25]:
def get_jaccard(x, y):
    x_sh = get_shingles(x, 10)
    y_sh = get_shingles(y, 10)

    return len(x_sh & y_sh) / len(x_sh | y_sh)

In [26]:
def jaccard_choose_parameters(k=5, num_hashes=100, bands=20):
    candidates = find_similar_strings(
        sents, k=k, num_hashes=num_hashes, bands=bands
    )
    jaccard_similarities = []
    for candidate in candidates:
        jaccard_similarities.append(
            get_jaccard(
                sents[candidate[0]],
                sents[candidate[1]],
            )
        )
        print(f"Sentence:\n{sents[candidate[0]]}\n")
        print(f"Potentially similar sentence:\n{sents[candidate[1]]}\n")
        print(f"Jaccard Similarity:\n{jaccard_similarities[-1]}\n\n")

    print("Minimal Jaccard Similarity:", min(jaccard_similarities))

## Эксперимент 1

In [27]:
# default setup
jaccard_choose_parameters()

  0%|          | 0/9340 [00:00<?, ?it/s]

0it [00:00, ?it/s]

Sentence:
– Очень рада вас видеть, – сказала княгиня.

Potentially similar sentence:
– Это только от скуки, – сказала княгиня.



Jaccard Similarity:
0.19298245614035087


Sentence:
– Ну, хорошо, хорошо.

Potentially similar sentence:
– Ну, хорошо, хорошо!..

Jaccard Similarity:
0.7333333333333333


Sentence:
– Я не понимаю этого.



Potentially similar sentence:
Я не понимаю, не понимаю этого!

Jaccard Similarity:
0.36


Sentence:
Срам просто!

– Ах, как неприятно! – сказала княгиня.

Potentially similar sentence:
– Это только от скуки, – сказала княгиня.



Jaccard Similarity:
0.14705882352941177


Sentence:
Алексей Александрович был в министерстве.

Potentially similar sentence:
Алексей Александрович был не ревнив.

Jaccard Similarity:
0.40476190476190477


Sentence:
Алексей Александрович взглянул на него.

Potentially similar sentence:
– Алексей Александрович!

Jaccard Similarity:
0.36363636363636365


Sentence:
– Видно, что ему жалко Пилата.



Potentially similar sentence:
Она ск

Эти гиперпараметры неоптимальны: среди кандидатов в дубликаты есть такие, мера Жаккара между которыми близка к 0.

## Эксперимент 2

In [28]:
jaccard_choose_parameters(num_hashes=200)

  0%|          | 0/9340 [00:00<?, ?it/s]

0it [00:00, ?it/s]

Sentence:
Я не могу жить с ним.

Potentially similar sentence:
Я не могу, не могу жить с ним.

Jaccard Similarity:
0.5


Sentence:
– Ну, хорошо, хорошо.

Potentially similar sentence:
– Ну, хорошо, хорошо!..

Jaccard Similarity:
0.7333333333333333


Sentence:
– «Пробовали: – хуже».

Potentially similar sentence:
– «Пробовали: – хуже».

Jaccard Similarity:
1.0


Sentence:
Степан Аркадьич улыбнулся.

Potentially similar sentence:
И Степан Аркадьич улыбнулся.

Jaccard Similarity:
0.8947368421052632


Sentence:
– Боже мой, что я сделал!

Potentially similar sentence:
«Боже мой, что я сделал!

Jaccard Similarity:
0.8235294117647058


Sentence:
Степан Аркадьич улыбнулся.

Potentially similar sentence:
Степан Аркадьич улыбнулся.

Jaccard Similarity:
1.0


Sentence:
Степан Аркадьич улыбнулся.

Potentially similar sentence:
И Степан Аркадьич улыбнулся.

Jaccard Similarity:
0.8947368421052632


Minimal Jaccard Similarity: 0.5


Этот результат лучше: в кандидаты на дубликаты не попадают пары предложений, мера Жаккара которых меньше 0,5.

## Эксперимент 3

In [29]:
jaccard_choose_parameters(bands=10)

  0%|          | 0/9340 [00:00<?, ?it/s]

0it [00:00, ?it/s]

Sentence:
Я не могу жить с ним.

Potentially similar sentence:
Я не могу, не могу жить с ним.

Jaccard Similarity:
0.5


Sentence:
– «Пробовали: – хуже».

Potentially similar sentence:
– «Пробовали: – хуже».

Jaccard Similarity:
1.0


Sentence:
Степан Аркадьич улыбнулся.

Potentially similar sentence:
И Степан Аркадьич улыбнулся.

Jaccard Similarity:
0.8947368421052632


Sentence:
– Боже мой, что я сделал!

Potentially similar sentence:
«Боже мой, что я сделал!

Jaccard Similarity:
0.8235294117647058


Sentence:
Степан Аркадьич улыбнулся.

Potentially similar sentence:
Степан Аркадьич улыбнулся.

Jaccard Similarity:
1.0


Sentence:
Степан Аркадьич улыбнулся.

Potentially similar sentence:
И Степан Аркадьич улыбнулся.

Jaccard Similarity:
0.8947368421052632


Minimal Jaccard Similarity: 0.5


Минимальная мера Жаккара среди пар кандидатов такая же, как в Эксперименте 2: 0,5.

Но в результаты, в отличие от Эксперимента 2, не попала пара с мерой Жаккара = 0.7(3):

```
Sentence:
– Ну, хорошо, хорошо.

Potentially similar sentence:
– Ну, хорошо, хорошо!..

Jaccard Similarity:
0.7333333333333333
```

## Эксперимент 4

In [30]:
jaccard_choose_parameters(bands=10, num_hashes=200)

  0%|          | 0/9340 [00:00<?, ?it/s]

0it [00:00, ?it/s]

Sentence:
– «Пробовали: – хуже».

Potentially similar sentence:
– «Пробовали: – хуже».

Jaccard Similarity:
1.0


Sentence:
Степан Аркадьич улыбнулся.

Potentially similar sentence:
Степан Аркадьич улыбнулся.

Jaccard Similarity:
1.0


Minimal Jaccard Similarity: 1.0


Эти гиперпараметры неоптимальны, потому что нашлись только полные дубликаты, но не near duplicates.

Самым успешным из четырех экспериментов можно считать Эксперимент 2.